In [51]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.dates import DateFormatter, AutoDateLocator
import plotly.express as px
import plotly.offline as offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots



# Define the directory where your ".dat" files are located
directory = '/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/10_2024_Data/Picarro_Raw'

In [52]:
# Initialize an empty list to store DataFrames
dfs = []

# Traverse the directory and its subdirectories
for root, dirs, files in os.walk(directory):
    # Filter files with ".dat" extension
    dat_files = glob.glob(os.path.join(root, '*.dat'))
    # Read each ".dat" file into a DataFrame and append it to the list
    for file in dat_files:
        # Read the data with fixed-width parsing
        df = pd.read_fwf(file)  # You might need to adjust this depending on the file format
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
Picarro_Raw = pd.concat(dfs, ignore_index=True)

# Assuming you have a DataFrame called df and the columns are 'date_column' and 'time_column'
# Replace 'date_column' and 'time_column' with the names of your columns
combined_datetime = Picarro_Raw['DATE'] + ' ' + Picarro_Raw['TIME']

# Convert the combined column to datetime format
Picarro_Raw['UTC'] = pd.to_datetime(combined_datetime)

Picarro_Raw = Picarro_Raw.sort_values(by='UTC')

In [53]:
# Names of Colunms
column_names = Picarro_Raw.columns
# Numer of column:
num_columns = Picarro_Raw.shape[1]
## Numer of rows:
num_rows = Picarro_Raw.shape[0]

# Print the column names and number of columns and rows:
print('Colunm Names: ', column_names)
print('Number of colunms: ', num_columns)
print('Number of rows: ', num_rows)

Colunm Names:  Index(['DATE', 'TIME', 'FRAC_DAYS_SINCE_JAN1', 'FRAC_HRS_SINCE_JAN1',
       'JULIAN_DAYS', 'EPOCH_TIME', 'ALARM_STATUS', 'INST_STATUS',
       'CavityPressure', 'CavityTemp', 'DasTemp', 'EtalonTemp', 'species',
       'OutletValve', 'CH4', 'CH4_dry', 'CO2', 'CO2_dry', 'h2o_reported',
       'ch4_base', 'ch4_pzt_std', 'co2_base', 'co2_pzt_std', 'wlm1_offset',
       'wlm2_offset', 'UTC'],
      dtype='object')
Number of colunms:  26
Number of rows:  1008034


In [54]:
# sort Dataframe
Picarro_Raw = Picarro_Raw.sort_values(by='UTC')

# export to CSV:
Picarro_Raw.to_csv('/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/10_2024_Data/Picarro_Raw/Picarro_Raw.csv', index=False)

In [55]:
# Measument time at Geomatikum
start_date = pd.to_datetime('2024-04-22 10:13:00.000')
end_date = pd.to_datetime('2024-05-02 9:40:00.000')

In [56]:
#Filter Measuments at the Geomatikum
Picarro_Raw_Geomatikum = Picarro_Raw[(Picarro_Raw['UTC'] >= start_date) & (Picarro_Raw['UTC'] <= end_date)]

In [57]:
# 2 point Calibration befor and after
Cal_Befor_Slope = 1.006235889211451 
Cal_Befor_Intercept = -0.06468001949787094

Cal_After_Slope = 1.006435416534788 
Cal_After_Intercept = -0.09883248115659171

Cal_Slope = (Cal_Befor_Slope + Cal_After_Slope)/2
Cal_Intercept = (Cal_Befor_Intercept + Cal_After_Intercept)/2

print('Calibration Slope: ', Cal_Slope)
print('Calibration Intercept: ', Cal_Intercept)

Calibration Slope:  1.0063356528731195
Calibration Intercept:  -0.08175625032723133


In [58]:
# Apply calibration factor to raw file
Picarro_Calibrated_Geomatikum = Picarro_Raw_Geomatikum.copy()
Picarro_Calibrated_Geomatikum['CH4_Calibrated'] = Picarro_Calibrated_Geomatikum['CH4'] * Cal_Slope + Cal_Intercept

#Export calibrated file to CSV
Picarro_Calibrated_Geomatikum.to_csv('/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/OutputData/2024_Picarro_Calibrated.csv', index=False)


In [59]:
# Import Geomatikum Wind Data
Geomatikum_Wind = pd.read_csv('/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/10_2024_Data/Uni_Hamburg/Geomatikum_Wind.csv')

# Import St. Pauli Water Level Data
WL_StPauli = pd.read_csv('/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/10_2024_Data/Water_Level_StPauli/WL_StPauli_merged.csv')

# Formate the date column to datetime
Geomatikum_Wind['UTC'] = pd.to_datetime(Geomatikum_Wind['UTC'])
WL_StPauli['UTC'] = pd.to_datetime(WL_StPauli['UTC'])

# WL_StPauli['UTC'] = WL_StPauli['UTC'].dt.tz_convert(None)
WL_StPauli['WL_Height'] = pd.to_numeric(WL_StPauli['WL_Height'], errors='coerce')

# Remove rows where 'numeric_column' contains non-numeric values (NaN)
WL_StPauli = WL_StPauli.dropna(subset=['WL_Height'])

# Filter Measuments during Geomatikum measuments
Geomatikum_Wind = Geomatikum_Wind[(Geomatikum_Wind['UTC'] >= start_date) & (Geomatikum_Wind['UTC'] <= end_date)]
WL_StPauli = WL_StPauli[(WL_StPauli['UTC'] >= start_date) & (WL_StPauli['UTC'] <= end_date)]

In [60]:
# Selct windoe size for roling average (Seconds)
window_size = 300

rolling_avg = []
rolling_CH4 = Picarro_Calibrated_Geomatikum['CH4_Calibrated'].rolling(window=window_size).mean()
# rolling_CO2 = Picarro_Calibrated_Geomatikum['CO2'].rolling(window=window_size).mean()
Picarro_Calibrated_Geomatikum['CH4_Roll_Avg'] = rolling_CH4
# Geomatikum_Picarro['CO2_Smoothed'] = rolling_CO2

In [61]:
# Set 'datetime_column' as the index
Picarro_Calibrated_Geomatikum.set_index('UTC', inplace=True)

# Resample the DataFrame to get the  10 minute averages
Geomatikum_Avg = Picarro_Calibrated_Geomatikum.resample('10T').mean()

# Reset the index if you want the datetime_column back as a regular column
Geomatikum_Avg.reset_index(inplace=True)


# Set 'datetime_column' as the index
WL_StPauli.set_index('UTC', inplace=True)

# Resample the DataFrame to get the 10 minute averages
WL_StPauli_Avg = WL_StPauli.resample('10T').mean()

# Reset the index if you want the datetime_column back as a regular column
WL_StPauli_Avg.reset_index(inplace=True)

Geomatikum = []
Geomatikum = pd.merge(Geomatikum_Avg, Geomatikum_Wind, on='UTC', how='outer')
Geomatikum = pd.merge(Geomatikum, WL_StPauli_Avg, on='UTC', how='outer')

#Export calibrated file to CSV
Picarro_Calibrated_Geomatikum.to_csv('/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/OutputData/2024_Geomatikum_Ch4_Wind_Waterlevel.csv', index=False)



In [62]:
# Create figure with subplots
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=('CH4 10 min Avg.', 'Water level hight (St. Pauli)','Wind Direction', 'Wind Speed'), vertical_spacing=0.05)

# Add trace for the first DataFrame to the first subplot
fig.add_trace(go.Scatter(x=Geomatikum['UTC'], y=Geomatikum['CH4'], mode='lines', name='CH4'), row=1, col=1)

# Add trace for the second DataFrame to the second subplot
fig.add_trace(go.Scatter(x=Geomatikum['UTC'], y=Geomatikum['WL_Height'], mode='lines', name='WL_Height'), row=2, col=1)

# Add trace for the second DataFrame to the second subplot
fig.add_trace(go.Scatter(x=Geomatikum['UTC'], y=Geomatikum['Direction'], mode='lines', name='Direction'), row=3, col=1)

fig.add_trace(go.Scatter(x=Geomatikum['UTC'], y=Geomatikum['Velocity'], mode='lines', name='Velocity'), row=4, col=1)

# Update layout
fig.update_layout(height=1200, width=1600, title_text="Geomatikum Plot", showlegend=False)

# Show figure
fig.show()
offline.plot(fig, filename='/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/OutputData/Plots/26_2024_Interactive_Geomatikum_Calibraed_Smoothed_Timeline_with_Wind_Waterlevel.html', auto_open=False)


'/Users/juanbettinelli/Documents/Uni/MasterThesis/4_Scrips_and_Data/4_Data/OutputData/Plots/26_2024_Interactive_Geomatikum_Calibraed_Smoothed_Timeline_with_Wind_Waterlevel.html'